In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import current_timestamp, lit

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
laptimes_schema  = StructType(fields = [
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [0]:
df_laptimes = spark.read \
    .option("header", True) \
    .schema(laptimes_schema) \
    .csv(f"{raw_folder_path}/{v_file_date}/lap_times/*.csv")

In [0]:
df_laptimes_renamed = df_laptimes.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("file_date", lit(v_file_date))


In [0]:
%run "../includes/common_functions"

In [0]:
merge_delta_data(df_laptimes_renamed, 'f1_processed','lap_times',processed_folder_path, 'tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id AND tgt.lap = src.lap','race_id')